In [1]:
import ray
ray.init(num_cpus=4)

2022-05-11 00:19:27,160	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8265
2022-05-11 00:19:27,191	WARNING services.py:1983 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 3221225472 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=6.39gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.8.12', ray_version='2.0.0.dev0', ray_commit='{{RAY_COMMIT_SHA}}', address_info={'node_ip_address': '10.42.3.57', 'raylet_ip_address': '10.42.3.57', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-05-11_00-19-17_748490_89/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-05-11_00-19-17_748490_89/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-05-11_00-19-17_748490_89', 'metrics_export_port': 63488, 'gcs_address': '10.42.3.57:65048', 'address': '10.42.3.57:65048', 'node_id': 'a9041c598135d1dddac0599d9c7753f1ecf18699e2656d126f6ab5b8'})

In [14]:
#tag::ray_metrics_singleton[]
# Singleton for reporting a Ray metric

class FailureCounter(object):
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            print('Creating the object')
            cls._instance = super(FailureCounter, cls).__new__(cls)
            from ray.util.metrics import Counter
            cls._instance.counter = Counter(
                "failure",
                description="Number of failures (goes up only).")
        return cls._instance

# This will fail with every zero because divide by zero
@ray.remote
def remote_fun(x):
    try:
        return 10 / x
    except:
        FailureCounter().counter.inc()
        return None
#end::ray_metrics_singleton[]

In [17]:
ray.get(remote_fun.remote(0))

In [32]:
#tag::ray_metrics_actor[]
# Singleton for reporting a Ray metric

@ray.remote
class MySpecialActor(object):
    def __init__(self, name):
        self.total = 0
        from ray.util.metrics import Counter, Gauge
        self.failed_withdrawls = Counter(
            "failed_withdrawls", description="Number of failed withdrawls.",
            tag_keys=("actor_name",), # Useful if you end up sharding actors
        )
        self.failed_withdrawls.set_default_tags({"actor_name": name})
        self.total_guage = Gauge(
            "money",
            description="How much money we have in total. Goes up and down.",
            tag_keys=("actor_name",), # Useful if you end up sharding actors
        )
        self.total_guage.set_default_tags({"actor_name": name})
        self.accounts = {}

    def deposit(self, account, amount):
        if account not in self.accounts:
            self.accounts[account] = 0
        self.accounts[account] += amount
        self.total += amount
        self.total_guage.set(self.total)
        
    def withdrawl(self, account, amount):
        if account not in self.accounts:
            self.failed_withdrawls.inc()
            raise Exception("No account")
        if self.accounts[account] < amount:
            self.failed_withdrawls.inc()
            raise Exception("Not enough money")
        self.accounts[account] -= amount
        self.total -= amount
        self.total_guage.set(self.total)
#end::ray_metrics_actor[]

In [33]:
bank1 = MySpecialActor.remote("bank1")

In [34]:
ray.get(bank1.deposit.remote("boo", 5))

In [35]:
ray.get(bank1.withdrawl.remote("holden", 5))

RayTaskError: [36mray::MySpecialActor.withdrawl()[39m (pid=1051, ip=10.42.3.57, repr=<__main__.MySpecialActor object at 0x7fac201400>)
  File "/tmp/ipykernel_89/651893236.py", line 32, in withdrawl
Exception: No account

In [36]:
ray.get(bank1.withdrawl.remote("boo", 5))